In [1]:
suppressMessages(library(repr))
options(repr.plot.width=25,
        repr.plot.height=10,
        #repr.plot.pointsize=50,
        repr.plot.family='serif'
       )

In [2]:
source("../../../src/Variacion/functions.R")

# Data variables

In [28]:
municipio = "Santander"
site = "" # c("es1580a", "es0118a")

data.by.file = TRUE

pollutant = c("o3")#, "no", "o3", "pm10", "pm2.5")
start_dt = 2010

# Lockdown dates

In [29]:
periods <- list(
    # lockdown
    lockdown = as.numeric(ymd_hms("2020-03-14 00:00:00")),
    #end.lockdown = as.numeric(ymd_hms("2020-04-28 00:00:00")),

    # poslockdown
    fases = as.numeric(ymd_hms("2020-05-01 00:00:00")),
    normalidad = as.numeric(ymd_hms("2020-06-21 00:00:00")),

    # New lockdown
    new.lockdown = as.numeric(ymd_hms("2020-10-25 00:00:00"))
)

---

In [30]:
if (site == "" && municipio != "") {
    site <- get.site.municipio(municipio,
                               fileName="../../../data/curation/sitesAQ.csv")
}

In [31]:
data.AQ <- get.AQdata(site, pollutant, start_dt, data.by.file)
head(data.AQ)

date,date_end,site,variable,process,summary,validity,unit,value
<dttm>,<dttm>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<dbl>


# Calculo Propio de Media diaria

In [32]:
data.AQ.dy <- data.AQ %>%
                group.dy(unit="day", FUN=mean)

ERROR: Error in aggregate.data.frame(as.data.frame(x), ...): no rows to aggregate


# Comparar Datos por Año

In [ ]:
data.all <- compare.years(data.AQ.dy, last.yr=2020)

# Variaciones diarias

In [ ]:
# Calcular variaciones
var.med <- get.difference(data.all, "2020",c("2019", 
                                             paste(start_dt, "-2019", sep="")))
var.med[,4:ncol(var.med)] <- var.med[,4:ncol(var.med)]*100

In [ ]:
head(var.med)

# Obtain Breakpoints from Variations

In [ ]:
var <- data.frame(y=var.med$"2010-2019.vs.2020",
                  x=as.numeric(var.med$date)
                 )

## Using `segmented` package

In [ ]:
suppressMessages(library(segmented))

my.seg <- segmented(lm(y ~ x + 1, data=var), 
                      seg.Z = ~ x,
                      psi = list(x = periods)
                     )

# Tendencia en los intervalos obtenidos

In [33]:
suppressMessages(library(mblm))
a <- mblm(y ~ x + 1,
          var[var$x < my.seg$psi[1],],
          repeated=T)

# Representar Datos

In [ ]:
plot <- ggplot(data=var, aes(x=as_datetime(x)))+ 
    scale_color_manual(name = "Years", 
                       values = c("2010-2019" = "red",
                                  "2019" = "blue"))

for (k in 1:(nrow(my.seg$psi))) {
    plot <- plot + geom_line(aes(y=y, color="2010-2019"), size=0.2)
    
    plot <- plot + geom_vline(xintercept = my.seg$psi[k,"Est."], 
                              linetype=5, color = "red", size=1.5)
    plot <- plot + geom_vline(xintercept = my.seg$psi[k,"Est."] - my.seg$psi[k,"St.Err"], 
                              linetype=5, color = "orange", size=1)
    plot <- plot + geom_vline(xintercept = my.seg$psi[k,"Est."] + my.seg$psi[k,"St.Err"], 
                              linetype=5, color = "orange", size=1)
}

for (k in 1:(length(periods))) {
    plot <- plot + geom_vline(xintercept = as.numeric(periods[k]), 
                              linetype=4, color = "black", size=1)
}


plot